# MidTerm_Assignment

### Problem_Statement

You are given the reviews dataset. These are 194439 amazon reviews for cell phones and
accessories taken from https://jmcauley.ucsd.edu/data/amazon/ Use the “reviewText” and
“overall” fields from this file. The goal is to predict the rating given the review by modeling it as
a multi-class classification problem.

3. Average of word embeddings
a. Learn word2vec models using gensim on this dataset with the following settings:
(a) Size=100, 200, 300, (b) Window=3,7, (c) Min_count=2, 5. Use skipgram.
i. This will give 12 word2vec models. For each of these models, for each
review take average word embeddings and train a logistic regression.
Report accuracy on test set.
b. Use the already available google word2vec model. For each review take average
word embeddings and train a logistic regression. Report accuracy on test set.
c. Use the already available glove models – 50D, 100D and 200D. For each review
take average word embeddings and train a logistic regression. Report accuracy on
test set for each of the three sized embeddings.

### Importing the Required Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import nltk
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.linear_model import LogisticRegression
from gensim.models import Word2Vec

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Download stopwords and tokenizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

Reading the data file

In [ ]:
# Load the entire dataset
full_data = pd.read_json("/content/drive/MyDrive/Mid_Term/dataset/Cell_Phones_and_Accessories_5.json", lines=True)

# Take a random sample of 5% of the data
sample_size = int(0.02 * len(full_data))
data = full_data.sample(n=sample_size, random_state=42)

In [ ]:
data.head()

reviewerID        asin       reviewerName helpful  \
156187  A1P3ACSVXZ1JUM  B00C0ZKI9Q              LO127  [0, 0]   
102252  A2Z3IZDQUIDRR9  B008FQVO5G            yogitim  [8, 8]   
23146   A3EOVXI1VZIHUQ  B003Y74AZ2  Z. Freeman "Zach"  [1, 2]   
86461    AWU526GBZ64MH  B007ZL0TJ4               Bree  [0, 0]   
62407   A3F7F7QKQP2FKT  B005VNK7T0             Beanie  [0, 0]   

                                               reviewText  overall  \
156187  iBOLT xProDock Active Car Dock/Holder/Mount fo...        5   
102252  This pouch is everything that I was looking fo...        5   
23146   First of all - the case I had on my iPhone 4 p...        3   
86461   I ordered this case because I had the same cas...        5   
62407   Bought a few of these as gifts. They were a bi...        5   

                                       summary  unixReviewTime   reviewTime  
156187                          Fits perfectly      1372032000  06 24, 2013  
102252                            Great Pouch!      1361577600  02 23, 2013  
23146   An average iPhone case - nothing fancy      1296604800   02 2, 2011  
86461                               Great Case      1379635200  09 20, 2013  
62407                                     Cute      1371859200  06 22, 2013

Preprocessing

In [ ]:
# Preprocess text data (tokenization, lowercasing, lemmatization, stopword removal)
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.isalpha() and word.lower() not in stop_words]
    return ' '.join(words)



In [ ]:
# Apply text preprocessing to the 'reviewText' column
data['reviewText'] = data['reviewText'].apply(preprocess_text)

In [ ]:
data['reviewText'] = data['reviewText'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
data['reviewText'] = data['reviewText'].apply(lambda x: re.sub(r'\d+', '', x))

In [ ]:
# Split data into train, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)


In [ ]:
# Load Google Word2Vec model
google_word2vec_path = "/content/drive/MyDrive/archive/GoogleNews-vectors-negative300.bin"
google_model = KeyedVectors.load_word2vec_format(google_word2vec_path, binary=True)



Word2Vec **models**

In [ ]:
# Task a: Learn and evaluate Word2Vec models with different settings
embedding_sizes = [100, 200, 300]
window_sizes = [3, 7]
min_counts = [2, 5]
results = {}

for size in embedding_sizes:
    for window in window_sizes:
        for min_count in min_counts:
            model = Word2Vec(
                sentences=[text.split() for text in data['reviewText']],
                vector_size=size,  # For newer gensim versions
                window=window,
                min_count=min_count,
                sg=1,  # Skip-gram
                epochs=10  # Number of iterations
            )
            X = np.array([
                np.mean([model.wv[word] for word in words if word in model.wv] or [np.zeros(size)], axis=0)
                for words in data['reviewText']
            ])
            X_train, X_test, y_train, y_test = train_test_split(X, data['overall'], test_size=0.2, random_state=42)
            classifier = LogisticRegression(max_iter=1000)
            classifier.fit(X_train, y_train)
            y_pred = classifier.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            results[(size, window, min_count)] = accuracy

# Print results for Task a
print("Results for Task a:")
for params, accuracy in results.items():
    size, window, min_count = params
    print(f"Embedding Size: {size}, Window Size: {window}, Min Count: {min_count}, Accuracy: {accuracy:.4f}")


Results for Task a:
Embedding Size: 100, Window Size: 3, Min Count: 2, Accuracy: 0.5643
Embedding Size: 100, Window Size: 3, Min Count: 5, Accuracy: 0.5643
Embedding Size: 100, Window Size: 7, Min Count: 2, Accuracy: 0.5643
Embedding Size: 100, Window Size: 7, Min Count: 5, Accuracy: 0.5643
Embedding Size: 200, Window Size: 3, Min Count: 2, Accuracy: 0.5643
Embedding Size: 200, Window Size: 3, Min Count: 5, Accuracy: 0.5643
Embedding Size: 200, Window Size: 7, Min Count: 2, Accuracy: 0.5643
Embedding Size: 200, Window Size: 7, Min Count: 5, Accuracy: 0.5643
Embedding Size: 300, Window Size: 3, Min Count: 2, Accuracy: 0.5643
Embedding Size: 300, Window Size: 3, Min Count: 5, Accuracy: 0.5643
Embedding Size: 300, Window Size: 7, Min Count: 2, Accuracy: 0.5643
Embedding Size: 300, Window Size: 7, Min Count: 5, Accuracy: 0.5643


**Google** **Word2Vec** model

In [ ]:
# Task b: Use Google Word2Vec model
X_google = np.array([np.mean([google_model[word] for word in words if word in google_model] or [np.zeros(300)], axis=0) for words in data['reviewText']])
X_train_google, X_test_google, y_train_google, y_test_google = train_test_split(X_google, data['overall'], test_size=0.2, random_state=42)
classifier_google = LogisticRegression(max_iter=1000)
classifier_google.fit(X_train_google, y_train_google)
y_pred_google = classifier_google.predict(X_test_google)
accuracy_google = accuracy_score(y_test_google, y_pred_google)
print("Accuracy using Google Word2Vec:", accuracy_google)



Accuracy using Google Word2Vec: 0.5642673521850899


## Already available glove models – 50D, 100D and 200D

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

# Paths to GloVe embeddings
glove_path_50 = "/content/drive/MyDrive/glove.6B/glove.6B.50d.txt"
glove_path_100 = "/content/drive/MyDrive/glove.6B/glove.6B.100d.txt"
glove_path_200 = "/content/drive/MyDrive/glove.6B/glove.6B.200d.txt"

# Convert GloVe format to Word2Vec format
glove_w2v_50 = "/content/drive/MyDrive/glove.6B/glove.6B.50d.word2vec.txt"
glove_w2v_100 = "/content/drive/MyDrive/glove.6B/glove.6B.100d.word2vec.txt"
glove_w2v_200 = "/content/drive/MyDrive/glove.6B/glove.6B.200d.word2vec.txt"

# Convert GloVe to Word2Vec format
glove2word2vec(glove_path_50, glove_w2v_50)
glove2word2vec(glove_path_100, glove_w2v_100)
glove2word2vec(glove_path_200, glove_w2v_200)

# Load GloVe embeddings in Word2Vec format
glove_model_50 = KeyedVectors.load_word2vec_format(glove_w2v_50, binary=False)
glove_model_100 = KeyedVectors.load_word2vec_format(glove_w2v_100, binary=False)
glove_model_200 = KeyedVectors.load_word2vec_format(glove_w2v_200, binary=False)

glove_models = {
    50: glove_model_50,
    100: glove_model_100,
    200: glove_model_200
}

glove_results = {}

for size, glove_model in glove_models.items():
    X_glove = np.array([
        np.mean([glove_model[word] for word in words if word in glove_model] or [np.zeros(size)], axis=0)
        for words in data['reviewText']
    ])
    X_train_glove, X_test_glove, y_train_glove, y_test_glove = train_test_split(X_glove, data['overall'], test_size=0.2, random_state=42)
    classifier_glove = LogisticRegression(max_iter=1000)
    classifier_glove.fit(X_train_glove, y_train_glove)
    y_pred_glove = classifier_glove.predict(X_test_glove)
    accuracy_glove = accuracy_score(y_test_glove, y_pred_glove)
    glove_results[size] = accuracy_glove

# Print results for Task c
print("Results for Task c:")
for size, accuracy in glove_results.items():
    print(f"GloVe Embedding Size: {size}, Accuracy: {accuracy:.4f}")


Results for Task c:
GloVe Embedding Size: 50, Accuracy: 0.5643
GloVe Embedding Size: 100, Accuracy: 0.5643
GloVe Embedding Size: 200, Accuracy: 0.5656
